# Lab 1: Matrix multiplication

In [106]:
import numpy as np 
import matplotlib.pyplot as plt
import time

### Classic $O(n^3$) method

In [107]:
def multiply_classic(M1, M2):
    m1, n1 = len(M1), len(M1[0])
    m2, n2 = len(M2), len(M2[0])

    return [
        [sum(M1[i][k] * M2[k][j] for k in range(n1)) for j in range(n2)]
        for i in range(m1)
    ]

### Binet's recursive method

In [108]:
def multiply_binet(M1, M2, count=None):
    # Counter for total number of operations
    if count is None: 
        count = {'add': 0, 'mul': 0}

    if not isinstance(M1, np.ndarray):
        M1 = np.array(M1)
    if not isinstance(M2, np.ndarray):
        M2 = np.array(M2)

    r1, c1 = M1.shape
    r2, c2 = M2.shape

    if c1 != r2:
        raise Exception("Invalid matrix sizes.")
    
    if r1 == 1 and c1 == 1 and c2 == 1:
        count['mul'] += 1
        return M1 @ M2, count
    
    if r1 == 1 or c1 == 1 or c2 == 1:
        if r1 == 1 and c1 == 1:
            count['mul'] += c2
        elif c2 == 1:
            count['mul'] += r1 * c1
            count['add'] += r1 * (c1 - 1)
        else:
            count['mul'] += r1 * c1 * c2
            count['add'] += r1 * (c1 - 1) * c2

        return M1 @ M2, count
    
    # Upper-left submatrices
    A1 = M1[:r1//2, :c1//2]
    A2 = M2[:r2//2, :c2//2]

    # Upper-right submatrices
    B1 = M1[:r1//2, c1//2:c1]
    B2 = M2[:r2//2, c2//2:c2]

    # Lower-left submatrices
    C1 = M1[r1//2:r1, :c1//2]
    C2 = M2[r2//2:r2, :c2//2]

    # Lower-right submatrices
    D1 = M1[r1//2:r1, c1//2:c1]
    D2 = M2[r2//2:r2, c2//2:c2]

    P1, count = multiply_binet(A1, A2, count)
    P2, count = multiply_binet(B1, C2, count)
    P3, count = multiply_binet(A1, B2, count)
    P4, count = multiply_binet(B1, D2, count)
    P5, count = multiply_binet(C1, A2, count)
    P6, count = multiply_binet(D1, C2, count)
    P7, count = multiply_binet(C1, B2, count)
    P8, count = multiply_binet(D1, D2, count)

    count["add"] += sum(i.size for i in [P1, P3, P5, P7])

    UL = P1 + P2
    UR = P3 + P4
    LL = P5 + P6
    LR = P7 + P8
 
    res = np.block(
        [[UL, UR],
        [LL, LR]]
    )

    return res, count

### Strassen's multiplication for square matrices of any size

In [109]:
def multiply_strassen(M1, M2, count=None):
    if count is None:
        count = {'add': 0, 'mul': 0}
    
    if not isinstance(M1, np.ndarray):
        M1 = np.array(M1)
    if not isinstance(M2, np.ndarray):
        M2 = np.array(M2)

    r1, c1 = M1.shape
    r2, c2 = M2.shape

    if c1 != r2:
        raise Exception("Invalid matrix sizes.")

    if r1 == 1 or c1 == 1 or c2 == 1:
        if r1 == 1 and c1 == 1:
            count['mul'] += c2
        elif c2 == 1:
            count['mul'] += r1 * c1
            count['add'] += r1 * (c1 - 1)
        else:
            count['mul'] += r1 * c1 * c2
            count['add'] += r1 * (c1 - 1) * c2
        return M1 @ M2, count
    
    if r1 % 2 == 0:
        # Upper-left submatrices 
        A1 = M1[:r1//2, :c1//2]
        A2 = M2[:r2//2, :c2//2]

        # Upper-right submatrices
        B1 = M1[:r1//2, c1//2:c1]
        B2 = M2[:r2//2, c2//2:c2]

        # Lower-left submatrices
        C1 = M1[r1//2:r1, :c1//2]
        C2 = M2[r2//2:r2, :c2//2]

        # Lower-right submatrices
        D1 = M1[r1//2:r1, c1//2:c1]
        D2 = M2[r2//2:r2, c2//2:c2]


        count['add'] += A1.size * 2
        P1, count = multiply_strassen(A1 + D1, A2 + D2, count)
        count['add'] += C1.size * 2
        P2, count = multiply_strassen(C1 + D1, A2, count)
        count['add'] += B2.size * 2
        P3, count = multiply_strassen(A1, B2 - D2, count)
        count['add'] += C2.size * 2
        P4, count = multiply_strassen(D1, C2 - A2, count)
        count['add'] += A1.size * 2
        P5, count = multiply_strassen(A1 + B1, D2, count)
        count['add'] += A1.size * 2
        P6, count = multiply_strassen(C1 - A1, A2 + B2, count)
        count['add'] += B1.size * 2
        P7, count = multiply_strassen(B1 - D1, C2 + D2, count)


        UL = P1 + P4 - P5 + P7
        UR = P3 + P5
        LL = P2 + P4 
        LR = P1 - P2 + P3 + P6

        count['add'] += (P1.size * 8)

    else:
        # Dividing matrices into:
        #   - Upper-left square block of even size (M)
        #   - Upper-right block (v): column of even size
        #   - Lower-left block (w): row of even size
        #   - Lower-right block (s): scalar (or 1x1 matrix)
        M1, v1, w1, s1 = M1[:r1-1, :c1-1], M1[:r1-1, c1-1:], M1[r1-1:, :c1-1], M1[r1-1:, c1-1:]
        M2, v2, w2, s2 = M2[:r2-1, :c2-1], M2[:r2-1, c2-1:], M2[r2-1:, :c2-1], M2[r2-1:, c2-1:]
    
        M1M2, count = multiply_strassen(M1, M2, count)

        UL = M1M2 + v1 @ w2
        UR = M1 @ v2 + v1 @ s2
        LL = w1 @ M2 + s1 @ w2
        LR = w1 @ v2 + s1 @ s2

        count["add"] += UL.size + UR.size + LL.size + LR.size     # during adding matrices
        count["add"] += M1.shape[0] + v1.size-1 + M2.shape[0] + w2.size-1 + w1.size-1  # during scalar product calculations

        count["mul"] += v1.size*w1.size + M1.size + v1.size + M2.size + w2.size + w1.size + 1


    res = np.block([
        [UL, UR],
        [LL, LR]
    ])

    return res, count
    

### Method from [Nature article](https://deepmind.google/discover/blog/discovering-novel-algorithms-with-alphatensor/#:~:text=In%20our%20paper,%20published%20today%20in%20Nature,%20we)

In [110]:
def multiply_nature(M1, M2):
    if not isinstance(M1, np.ndarray):
        M1 = np.array(M1)
    if not isinstance(M2, np.ndarray):
        M2 = np.array(M2)

    m1, n1 = M1.shape
    m2, n2 = M2.shape

    if m1 != 4 or not (n1 == m2 == n2 == 5):
        raise Exception("Invalid matrix sizes: should be (4x5)*(5x5)")
    
    h1 = M1[2][1] * (-M2[1][0] - M2[1][4] - M2[2][0])
    h2 = (M1[1][1] + M1[1][4] - M1[2][4]) * (-M2[1][4] - M2[4][0])
    h3 = (-M1[2][0] - M1[3][0] + M1[3][1]) * (-M2[0][0] + M2[1][4])
    h4 = (M1[0][1] + M1[0][3] + M1[2][3]) * (-M2[1][4] - M2[3][0])
    h5 = (M1[0][4] + M1[1][1] + M1[1][4]) * (-M2[1][3] + M2[4][0])
    h6 = (-M1[1][1] - M1[1][4] - M1[3][4]) * (M2[1][2] + M2[4][0])
    h7 = (-M1[0][0] + M1[3][0] - M1[3][1]) * (M2[0][0] + M2[1][3])
    h8 = (M1[2][1] - M1[2][2] - M1[3][2]) * (-M2[1][2] + M2[2][0])
    h9 = (-M1[0][1] - M1[0][3] + M1[3][3]) * (M2[1][2] + M2[3][0])
    h10 = (M1[1][1] + M1[1][4]) * M2[4][0]

    h11 = (-M1[1][0] - M1[3][0] + M1[3][1]) * (-M2[0][0] + M2[1][1])
    h12 = (M1[3][0] - M1[3][1]) * M2[0][0]
    h13 = (M1[0][1] + M1[0][3] + M1[1][3]) * (M2[1][1] + M2[3][0])
    h14 = (M1[0][2] - M1[2][1] + M1[2][2]) * (M2[1][3] + M2[2][0])
    h15 = (-M1[0][1] - M1[0][3]) * M2[3][0]
    h16 = (-M1[2][1] + M1[2][2]) * M2[2][0]
    h17 = (M1[0][1] + M1[0][3] - M1[1][0] + M1[1][1] - M1[1][2] + M1[1][3] - M1[2][1] + M1[2][2] - M1[3][0] + M1[3][1]) * M2[1][1]
    h18 = M1[1][0] * (M2[0][0] + M2[0][1] + M2[4][1])
    h19 = -M1[1][2] * (M2[2][0] + M2[2][1] + M2[4][1])
    h20 = (-M1[0][4] + M1[1][0] + M1[1][2] - M1[1][4]) * (-M2[0][0] - M2[0][1] + M2[0][3] - M2[4][1])

    h21 = (M1[1][0] + M1[1][2] - M1[1][4]) * M2[4][1]
    h22 = (M1[0][2] - M1[0][3] - M1[1][3]) * (M2[0][0] + M2[0][1] - M2[0][3] - M2[2][0] - M2[2][1] + M2[2][3] + M2[3][3])
    h23 = M1[0][2] * (-M2[2][0] + M2[2][3] + M2[3][3])
    h24 = M1[0][4] * (-M2[3][3] - M2[4][0] + M2[4][3])
    h25 = -M1[0][0] * (M2[0][0] - M2[0][3])
    h26 = (-M1[0][2] + M1[0][3] + M1[0][4]) * M2[3][3]
    h27 = (M1[0][2] - M1[2][0] + M1[2][2]) * (M2[0][0] - M2[0][3] + M2[0][4] + M2[2][4])
    h28 = -M1[2][3] * (-M2[2][4] - M2[3][0] - M2[3][4])
    h29 = M1[2][0] * (M2[0][0] + M2[0][4] + M2[2][4])
    h30 = (M1[2][0] - M1[2][2] + M1[2][3]) * M2[2][4]

    h31 = (-M1[0][3] - M1[0][4] - M1[2][3]) * (-M2[3][3] - M2[4][0] + M2[4][3] - M2[4][4])
    h32 = (M1[1][0] + M1[3][0] + M1[3][3]) * (M2[0][2] - M2[3][0] - M2[3][1] - M2[3][2])
    h33 = M1[3][2] * (-M2[2][0] - M2[2][2])
    h34 = M1[3][3] * (-M2[0][2] + M2[3][0] + M2[3][2])
    h35 = -M1[3][4] * (M2[0][2] + M2[4][0] + M2[4][2])
    h36 = (M1[1][2] - M1[1][4] - M1[3][4]) * (M2[2][0] + M2[2][1] + M2[2][2] + M2[4][1])
    h37 = (-M1[3][0] - M1[3][3] + M1[3][4]) * M2[0][2]
    h38 = (-M1[1][2] - M1[2][0] + M1[2][2] - M1[2][3]) * (M2[2][4] + M2[3][0] + M2[3][1] + M2[3][4])
    h39 = (-M1[2][0] - M1[3][0] - M1[3][3] + M1[3][4]) * (M2[0][2] + M2[4][0] + M2[4][2] + M2[4][4])
    h40 = (-M1[0][2] + M1[0][3] + M1[0][4] - M1[3][3]) * (-M2[2][0] - M2[2][2] + M2[2][3] + M2[3][3])

    h41 = (-M1[0][0] + M1[3][0] - M1[3][4]) * (M2[0][2] + M2[2][0] + M2[2][2] - M2[2][3] + M2[4][0] + M2[4][2] - M2[4][3])
    h42 = (-M1[1][0] + M1[1][4] - M1[2][4]) * (-M2[0][0] - M2[0][1] - M2[0][4] + M2[3][0] + M2[3][1] + M2[3][4] - M2[4][1])
    h43 = M1[1][3] * (M2[3][0] + M2[3][1])
    h44 = (M1[1][2] + M1[2][1] - M1[2][2]) * (M2[1][1] - M2[2][0])
    h45 = (-M1[2][2] + M1[2][3] - M1[3][2]) * (M2[2][4] + M2[3][0] + M2[3][2] + M2[3][4] + M2[4][0] + M2[4][2] + M2[4][4])
    h46 = -M1[2][4] * (-M2[4][0] - M2[4][4])
    h47 = (M1[1][0] - M1[1][4] - M1[2][0] + M1[2][4]) * (M2[0][0] + M2[0][1] + M2[0][4] - M2[3][0] - M2[3][1] - M2[3][4])
    h48 = (-M1[1][2] + M1[2][2]) * (M2[1][1] + M2[2][1] + M2[2][4] + M2[3][0] + M2[3][1] + M2[3][4])
    h49 = (-M1[0][0] - M1[0][2] + M1[0][3] + M1[0][4] - M1[1][0] - M1[1][2] + M1[1][3] + M1[1][4]) * (-M2[0][0] - M2[0][1] + M2[0][3])
    h50 = (-M1[0][3] - M1[1][3]) * (M2[1][1] - M2[2][0] - M2[2][1] + M2[2][3] - M2[3][1] + M2[3][3])

    h51 = M1[1][1] * (M2[1][0] + M2[1][1] - M2[4][0])
    h52 = M1[3][1] * (M2[0][0] + M2[1][0] + M2[1][2])
    h53 = -M1[0][1] * (-M2[1][0] + M2[1][3] + M2[3][0])
    h54 = (M1[0][1] + M1[0][3] - M1[1][1] - M1[1][4] - M1[2][1] + M1[2][2] - M1[3][1] + M1[3][2] - M1[3][3] - M1[3][4]) * M2[1][2]
    h55 = (M1[0][3] - M1[3][3]) * (-M2[1][2] + M2[2][0] + M2[2][2] - M2[2][3] + M2[3][2] - M2[3][3])
    h56 = (M1[0][0] - M1[0][4] - M1[3][0] + M1[3][4]) * (M2[2][0] + M2[2][2] - M2[2][3] + M2[4][0] + M2[4][2] - M2[4][3])
    h57 = (-M1[2][0] - M1[3][0]) * (-M2[0][2] - M2[0][4] - M2[1][4] - M2[4][0] - M2[4][2] - M2[4][4])
    h58 = (-M1[0][3] - M1[0][4] - M1[2][3] - M1[2][4]) * (-M2[4][0] + M2[4][3] - M2[4][4])
    h59 = (-M1[2][2] + M1[2][3] - M1[3][2] + M1[3][3]) * (M2[3][0] + M2[3][2] + M2[3][4] + M2[4][0] + M2[4][2] + M2[4][4])
    h60 = (M1[1][4] + M1[3][4]) * (M2[1][2] - M2[2][0] - M2[2][1] - M2[2][2] - M2[4][1] - M2[4][2])

    h61 = (M1[0][3] + M1[2][3]) * (M2[0][0] - M2[0][3] + M2[0][4] - M2[1][4] - M2[3][3] + M2[3][4] - M2[4][0] + M2[4][3] - M2[4][4])
    h62 = (M1[1][0] + M1[3][0]) * (M2[0][1] + M2[0][2] + M2[1][1] - M2[3][0] - M2[3][1] - M2[3][2])
    h63 = (-M1[2][2] - M1[3][2]) * (-M2[1][2] - M2[2][2] - M2[2][4] - M2[3][0] - M2[3][2] - M2[3][4])
    h64 = (M1[0][0] - M1[0][2] - M1[0][3] + M1[2][0] - M1[2][2] - M1[2][3]) * (M2[0][0] - M2[0][3] + M2[0][4])
    h65 = (-M1[0][0] + M1[3][0]) * (-M2[0][2] + M2[0][3] + M2[1][3] - M2[4][0] - M2[4][2] + M2[4][3])
    h66 = (M1[0][0] - M1[0][1] + M1[0][2] - M1[0][4] - M1[1][1] - M1[1][4] - M1[2][1] + M1[2][2] - M1[3][0] + M1[3][1]) * M2[1][3]
    h67 = (M1[1][4] - M1[2][4]) * (M2[0][0] + M2[0][1] + M2[0][4] - M2[1][4] - M2[3][0] - M2[3][1] - M2[3][4] + M2[4][1] + M2[4][4])
    h68 = (M1[0][0] + M1[0][2] - M1[0][3] - M1[0][4] - M1[3][0] - M1[3][2] + M1[3][3] + M1[3][4]) * (-M2[2][0] - M2[2][2] + M2[2][3])
    h69 = (-M1[0][2] + M1[0][3] - M1[1][2] + M1[1][3]) * (-M2[1][3] - M2[2][0] - M2[2][1] + M2[2][3] - M2[4][1] + M2[4][3])
    h70 = (M1[1][2] - M1[1][4] + M1[3][2] - M1[3][4]) * (-M2[2][0] - M2[2][1] - M2[2][2])

    h71 = (-M1[2][0] + M1[2][2] - M1[2][3] + M1[2][4] - M1[3][0] + M1[3][2] - M1[3][3] + M1[3][4]) * (-M2[4][0] - M2[4][2] - M2[4][4])
    h72 = (-M1[1][0] - M1[1][3] - M1[3][0] - M1[3][3]) * (M2[3][0] + M2[3][1] + M2[3][2])
    h73 = (M1[0][2] - M1[0][3] - M1[0][4] + M1[1][2] - M1[1][3] - M1[1][4]) * (M2[0][0] + M2[0][1] - M2[0][3] + M2[1][3] + M2[4][1] - M2[4][3])
    h74 = (M1[1][0] - M1[1][2] + M1[1][3] - M1[2][0] + M1[2][2] - M1[2][3]) * (M2[3][0] + M2[3][1] + M2[3][4])
    h75 = -(M1[0][1] + M1[0][3] - M1[1][1] - M1[1][4] - M1[2][0] + M1[2][1] + M1[2][3] + M1[2][4] - M1[3][0] + M1[3][1]) * M2[1][4]
    h76 = (M1[0][2] + M1[2][2]) * (-M2[0][0] + M2[0][3] - M2[0][4] + M2[1][3] + M2[2][3] - M2[2][4])


    c11 = -h10 + h12 + h14 - h15 - h16 + h53 + h5 - h66 - h7
    c21 = h10 + h11 - h12 + h13 + h15 + h16 - h17 - h44 + h51
    c31 = h10 - h12 + h15 + h16 - h1 + h2 + h3 - h4 + h75
    c41 = -h10 + h12 - h15 - h16 + h52 + h54 - h6 - h8 + h9

    c12 = h13 + h15 + h20 + h21 - h22 + h23 + h25 - h43 + h49 + h50
    c22 = -h11 + h12 - h13 - h15 - h16 + h17 + h18 - h19 - h21 + h43 + h44
    c32 = -h16 - h19 - h21 - h28 - h29 - h38 + h42 + h44 - h47 + h48
    c42 = h11 - h12 - h18 + h21 - h32 + h33 - h34 - h36 + h62 - h70

    c13 = h15 + h23 + h24 + h34 - h37 + h40 - h41 + h55 - h56 - h9
    c23 = -h10 + h19 + h32 + h35 + h36 + h37 - h43 - h60 - h6 - h72
    c33 = -h16 - h28 + h33 + h37 - h39 + h45 - h46 + h63 - h71 - h8
    c43 = h10 + h15 + h16 - h33 + h34 - h35 - h37 - h54 + h6 + h8 - h9

    c14 = -h10 + h12 + h14 - h16 + h23 + h24 + h25 + h26 + h5 - h66 - h7
    c24 = h10 + h18 - h19 + h20 - h22 - h24 - h26 - h5 - h69 + h73
    c34 = -h14 + h16 - h23 - h26 + h27 + h29 + h31 + h46 - h58 + h76
    c44 = h12 + h25 + h26 - h33 - h35 - h40 + h41 + h65 - h68 - h7

    c15 = h15 + h24 + h25 + h27 - h28 + h30 + h31 - h4 + h61 + h64
    c25 = -h10 - h18 - h2 - h30 - h38 + h42 - h43 + h46 + h67 + h74
    c35 = -h10 + h12 - h15 + h28 + h29 - h2 - h30 - h3 + h46 + h4 - h75
    c45 = -h12 - h29 + h30 - h34 + h35 + h39 + h3 - h45 + h57 + h59
    

    res = np.array(
        [
            [c11, c12, c13, c14, c15],
            [c21, c22, c23, c24, c25],
            [c31, c32, c33, c34, c35],
            [c41, c42, c43, c44, c45]
        ]
    )

    return res

### Comparision of runtime

In [111]:
matrices = np.load("matrices.npz") # if doesn't exist: run test_generator.py

In [ ]:
results = []
counts = []
times = []

for n in range(1, 25):
    print(f"size = {n}")
    A = matrices[f"A_{n}"]
    B = matrices[f"B_{n}"]
    
    start = time.perf_counter()
    res, cnt = multiply_binet(A, B)
    tm = time.perf_counter() - start

    results.append(res)
    counts.append(cnt)
    times.append(time)
    print(f"{tm:.5f}")